In [0]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [0]:
import argparse
import os
import numpy as np
import math
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

In [0]:
epochs = 300
batch_size = 32
learn_rate = 0.001
image_shape = (3, 200, 180)
interval = 100
beta1 = 0.5
beta2 = 0.999
latentFeaturesShape = 200
os.makedirs("1st_question", exist_ok=True)

In [0]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_features, out_features, normalize=True):
            layers = [nn.Linear(in_features, out_features)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_features, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latentFeaturesShape, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(image_shape))),
            nn.Tanh()
        )
        self.avgpool =  nn.AdaptiveAvgPool2d(1)

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *image_shape)
        return img

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(image_shape)), 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        
        img_flat = img.view(img.shape[0], -1)
        checkValidity = self.model(img_flat)

        return checkValidity

In [0]:
GAN_loss = torch.nn.BCELoss()

In [0]:
generator = Generator()
discriminator = Discriminator()

In [0]:
GeneratorOptimizer = torch.optim.Adam(generator.parameters(), lr=learn_rate, betas=(beta1, beta2))
DiscriminatorOptimizer = torch.optim.Adam(discriminator.parameters(), lr=learn_rate, betas=(beta1, beta2))

In [0]:
Tensor = torch.FloatTensor

In [0]:
root_dir = 'drive/My Drive/faces94'
files = glob(f"{root_dir}/**/**/*.jpg")

In [0]:
vam = {
    "female": 0,
    "male":1,
    "malestaff": 2 
}

In [0]:
transformLoader = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

In [0]:
All_Images = torch.stack([transformLoader(Image.open(x)) for x in files])
All_Labels = torch.tensor([vam[x.split('/')[-3]] for x in files])

In [0]:
def getDataloaders(All_Images,All_Labels):
  train_data, test_data = train_test_split(range(len(All_Images)), test_size=0.2, random_state=102)
  train_images = All_Images[train_data]
  train_label = All_Labels[train_data]
  test_images = All_Images[test_data]
  test_label = All_Labels[test_data]
  train_data = TensorDataset(train_images, train_label)
  test_data = TensorDataset(test_images, test_label)
  sampleTrain = RandomSampler(train_data)
  sampleTest = SequentialSampler(test_data)
  trainDataLoader = DataLoader(train_data, sampler=sampleTrain, batch_size=32)
  testDataLoader = DataLoader(test_data, sampler=sampleTest, batch_size=32)
  
  return trainDataLoader,testDataLoader

In [0]:
trainDataLoader,testDataLoader=getDataloaders(All_Images,All_Labels)

In [0]:
def run_model(epochs,trainDataLoader,testDataLoader,latentFeaturesShape,GeneratorOptimizer,DiscriminatorOptimizer):
  for epoch in range(epochs):
      for i, (images,_) in enumerate(trainDataLoader):
          valid = Variable(Tensor(images.size(0), 1).fill_(1.0), requires_grad=False)
          fake = Variable(Tensor(images.size(0), 1).fill_(0.0), requires_grad=False)
          trueImages = Variable(images.type(Tensor))
          GeneratorOptimizer.zero_grad()
          z = Variable(Tensor(np.random.normal(0, 1, (images.shape[0], latentFeaturesShape))))
          generatedImages = generator(z)
          GeneratorImagesLoss = GAN_loss(discriminator(generatedImages), valid)
          GeneratorImagesLoss.backward()
          GeneratorOptimizer.step()
          DiscriminatorOptimizer.zero_grad()
          real_loss = GAN_loss(discriminator(trueImages), valid)
          fake_loss = GAN_loss(discriminator(generatedImages.detach()), fake)
          DiscriminatorLoss = (real_loss + fake_loss) / 2
          DiscriminatorLoss.backward()
          DiscriminatorOptimizer.step()

          print(
              "{ Epoch --> %d of %d}  { Batch --> %d of %d }  { Generator Loss --> %f }  { Discriminator Loss --> %f }"
              % (epoch+1, epochs, i+1, len(trainDataLoader),GeneratorImagesLoss.item(),DiscriminatorLoss.item())
          )

          FinishedBatches = epoch * len(trainDataLoader) + i
          if FinishedBatches % interval == 0:
              save_image(generatedImages.data[:25], "1st_question/%d.png" % FinishedBatches, nrow=5, normalize=True)  
  

In [0]:
run_model(epochs,trainDataLoader,testDataLoader,latentFeaturesShape,GeneratorOptimizer,DiscriminatorOptimizer)

{ Epoch --> 1 of 1}  { Batch --> 1 of 77 }  { Generator Loss --> 0.681613 }  { Discriminator Loss --> 0.696671 }
{ Epoch --> 1 of 1}  { Batch --> 2 of 77 }  { Generator Loss --> 0.738853 }  { Discriminator Loss --> 0.324720 }
{ Epoch --> 1 of 1}  { Batch --> 3 of 77 }  { Generator Loss --> 0.666741 }  { Discriminator Loss --> 0.361286 }
{ Epoch --> 1 of 1}  { Batch --> 4 of 77 }  { Generator Loss --> 0.732876 }  { Discriminator Loss --> 0.332332 }
{ Epoch --> 1 of 1}  { Batch --> 5 of 77 }  { Generator Loss --> 1.037325 }  { Discriminator Loss --> 0.229643 }
{ Epoch --> 1 of 1}  { Batch --> 6 of 77 }  { Generator Loss --> 2.544907 }  { Discriminator Loss --> 0.041560 }
{ Epoch --> 1 of 1}  { Batch --> 7 of 77 }  { Generator Loss --> 3.707690 }  { Discriminator Loss --> 15.920053 }
{ Epoch --> 1 of 1}  { Batch --> 8 of 77 }  { Generator Loss --> 0.000183 }  { Discriminator Loss --> 4.432014 }
{ Epoch --> 1 of 1}  { Batch --> 9 of 77 }  { Generator Loss --> 0.000163 }  { Discriminator Lo